# Visualizing tools’ results

graph stuff:
- https://holoviews.org/gallery/demos/bokeh/network_graph.html
- https://holoviews.org/reference/elements/bokeh/Graph.html

In [ ]:
import holoviews as hv

from hv_anndata import ACCESSOR as A
from hv_anndata import register
from hv_anndata.interface import AdPath

register()

hv.extension("bokeh")

In [ ]:
from typing import Literal

import numpy as np
import pandas as pd
import scanpy as sc
from anndata import AnnData
from scanpy._utils import NeighborsView

In [ ]:
adata = sc.datasets.pbmc68k_reduced()
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

{func}`scanpy.pl.embedding` (i.e. {func}`scanpy.pl.pca`, {func}`scanpy.pl.umap`, {func}`scanpy.pl.diffmap`, {func}`scanpy.pl.tsne`, {func}`scanpy.pl.draw_graph`)

missing:
- `add_outline`
- fast graph routines: https://github.com/holoviz/holoviews/issues/6667
- `legend_position` doesn’t work for graphs

missing convenience:
- `groups` to restrict to a subset easily

In [ ]:
# without edges, see `scatter` in Basic notebook


def draw_graph(
    adata: AnnData,
    node_vdims: AdPath | list[AdPath] | None = None,
    edge_vdim: Literal["distances", "connectivities"] = "connectivities",
    *,
    neighbors_key: str = "neighbors",
) -> hv.Graph:
    adata = adata.copy()
    adata.obs["cell index"] = range(adata.n_obs)
    # TODO: allow to use things like A.obsp["connectivities"]  # noqa: TD003
    ns = NeighborsView(adata, neighbors_key)
    coo = ns[edge_vdim].tocoo()
    nodes = hv.Nodes(
        adata,
        [A.obsm["X_umap"][0], A.obsm["X_umap"][1], A.obs["cell index"]],
        node_vdims,
    )
    return hv.Graph(((*coo.coords, coo.data), nodes), vdims=edge_vdim)


draw_graph(adata, [A.obs["bulk_labels"]]).opts(
    node_color=A.obs["bulk_labels"],
    node_cmap="tab10",
    edge_color="connectivities",
    aspect="square",
    legend_position="right",
)

{func}`scanpy.pl.ranking` ({func}`scanpy.pl.pca_loadings`, {func}`scanpy.pl.pca_variance_ratio`)

missing:
- `xoffset`/`yoffset` taken from dimension or `text_xoffset`/`text_yoffset` available for `Labels`: https://github.com/holoviz/holoviews/issues/3884 

In [ ]:
def ranking(
    adata, dim: AdPath, n_points: int = 10, *, include_lowest: bool = True
) -> hv.Labels:
    [ax] = dim.axes
    # full arrays
    scores = dim(adata)
    labels = getattr(adata, ax).index

    # subset
    idx = np.argsort(scores)
    idx_top, idx_bot = idx[-n_points:][::-1], idx[:n_points][::-1]
    scores = np.r_[scores[idx_top], np.nan, scores[idx_bot]]
    labels = np.r_[labels[idx_top], ["⋯"], labels[idx_bot]]

    # prepare
    data = pd.DataFrame(
        dict(
            rank=np.arange(n_points * 2 + 1),
            score=np.where(np.isnan(scores), np.nanmean(scores), scores),
            dot=np.r_[[True] * n_points, False, [True] * n_points],
            text=labels,
            align=np.r_[["right"] * n_points, ["center"], ["left"] * n_points],
        )
    )
    if not include_lowest:
        data = data[:n_points]

    return hv.Labels(data, ["rank", "score"], ["text", "align"]).opts(
        angle=90, text_align="align", xticks=0
    ) * hv.Points(data, ["rank", "score"], ["text", "dot"]).opts(alpha="dot")


ranking(adata, A.varm["PCs"][0]).opts(aspect=1.2)

{func}`scanpy.pl.pca_overview` is just a layout of multiple `pca_` plots

{func}`scanpy.pl.embedding_density`

TODO: maybe better approach: https://holoviews.org/gallery/demos/bokeh/iris_density_grid.html

In [ ]:
sc.tl.embedding_density(adata, basis="umap", groupby="phase")

In [ ]:
hv.Scatter(
    adata,
    A.obsm["X_umap"][0],
    [A.obsm["X_umap"][1], A.obs["phase"], A.obs["umap_density_phase"]],
).opts(
    color=A.obs["umap_density_phase"], aspect="square", legend_position="right"
).groupby(A.obs["phase"], hv.NdLayout)